### Libraries, options

In [ ]:
import numpy as np
import os
import pandas as pd
import patsy
import re
import statsmodels.api as sm

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)

### Read data

In [ ]:
path_to_data = "path/to/data"
data = pd.read_csv(os.path.join(path_to_data, "survey-exp-data-corona.csv"), low_memory = False)
data = data.drop(columns = ["Unnamed: 0"])
print(len(data))
data.head()

### Check comprehension question

In [ ]:
data["comprehension_q"] = data["comprehension_q"].str.lower()

In [ ]:
def check_control(row):
    entry = row['comprehension_q']
    pattern = "(corona|corono|rona|virus|vairus|verious|covid|co vid|codv|19|\
disease|desisea|outbreak|pandemic|pandemoic|pndemic|epidemic)"
    if re.search(pattern, entry):
        return 1
    else:
        return 0
    
data['control_check'] = data.apply(check_control, axis = 1)
data[data['condition'] == 'corona_control'][['condition', 
                                             'control_check',
                                             'comprehension_q']].groupby(["condition", 
                                                                          "control_check"]).count()

In [ ]:
data[(data['condition'] == 'corona_control') & (data['control_check'] == 0)]

In [ ]:
def check_natural_ineq(row):
    entry = row['comprehension_q']
    pattern = "(elderly|ederly|elderley|elder|old|older|0lder|okder|weak|65|\
underlying|undsrline|medical|conditions|at risk|at most risk|higher risk|greater risk|extra care|\
susceptible|different|differently|differentky|affected most|most affected|affected the worst|\
vulnerability|valuable|vulable|vulnerable|certain populations|certain groups|same|heart|specific groups|\
some people more|poor health|hi risk|disproportionately|certain people|risk|people it affects|most|age|\
not affect|higher|group|affected|who|dffetenty|varing|more likely|affecting|diseased people|health|comorbidities|\
disproportionate|people infected|morbidities|infected by|differ|previous|with disease|disproportion)"
    if re.search(pattern, entry):
        return 1
    else:
        return 0
    
data['natural_ineq_check'] = data.apply(check_natural_ineq, axis = 1)
data[data['condition'] == 'corona_natural_ineq'][['condition', 
                                                  'natural_ineq_check',
                                                  'comprehension_q']].groupby(["condition", 
                                                                               "natural_ineq_check"]).count()

In [ ]:
data[(data['condition'] == 'corona_natural_ineq') & (data['natural_ineq_check'] == 0)]

In [ ]:
def check_class_ineq(row):
    entry = row['comprehension_q']
    pattern = "(poor|minority|minorities|low|black|hispanic|latin|color|same|\
different|differently|who|community|some people|racism|certain groups|certain races|race|\
groups|ethnic|sertin|differant|communities|miniorities|disproportional|certain|differntly|racial|\
inequality|equal|disproportionate|minorites|poc|less fortunate|not affected everyone|demog|comunitty|\
disproportion|most likely|group|isn’t affecting everyone|highly effected|surtane|affected|affecting|hard|\
proportion|affects|death rate|at risk)"
    if re.search(pattern, entry):
        return 1
    else:
        return 0
    
data['class_ineq_check'] = data.apply(check_class_ineq, axis = 1)
data[data['condition'] == 'corona_class_ineq'][['condition', 
                                                'class_ineq_check',
                                                'comprehension_q']].groupby(["condition", 
                                                                             "class_ineq_check"]).count()

In [ ]:
data[(data['condition'] == 'corona_class_ineq') & (data['class_ineq_check'] == 0)]

### Fit models

In [ ]:
temp = data[(data['control_check'] == 1) | (data['natural_ineq_check'] == 1) | (data['class_ineq_check'] == 1)]
print(temp.shape)
temp.head()

##### Figure 1

In [ ]:
y, X = patsy.dmatrices('coronavirus_serious_threat ~ C(condition) - 1', temp, return_type = 'dataframe')
ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices('must_save_economy ~ C(condition) - 1', temp, return_type = 'dataframe')
ols = sm.OLS(y, X)
ols.fit().summary()

##### Figure 2

In [ ]:
def get_at_risk(row):
    entry1 = row['self_at_risk']
    entry2 = row['family_member_at_risk']
    if entry1 + entry2 > 0:
        return 1
    else:
        return 0

temp['at_risk'] = temp.apply(get_at_risk, axis = 1)
temp.head()

In [ ]:
y, X = patsy.dmatrices('coronavirus_serious_threat ~ C(condition) - 1', 
                       temp[temp['at_risk'] == 1], 
                       return_type = 'dataframe')
ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices('coronavirus_serious_threat ~ C(condition) - 1', 
                       temp[temp['at_risk'] == 0], 
                       return_type = 'dataframe')
ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices('must_save_economy ~ C(condition) - 1', 
                       temp[temp['at_risk'] == 1], 
                       return_type = 'dataframe')
ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices('must_save_economy ~ C(condition) - 1', 
                       temp[temp['at_risk'] == 0], 
                       return_type = 'dataframe')
ols = sm.OLS(y, X)
ols.fit().summary()